In [1]:
import torch

from config import RAW_DATA_DIR

2025-07-18 07:57:26.973 | INFO     | config:<module>:9 - PROJ_ROOT path is: /home/arys/projects/unsupervised_ml_experimentation


In [3]:
input_tensor = torch.tensor([3.])

w1 = torch.tensor([1.], requires_grad=True)

w_mu = torch.tensor([2.], requires_grad=True)
w_std = torch.tensor([3.], requires_grad=True)

enc_out = input_tensor * w1
mu = w_mu * enc_out
std = w_std * enc_out
#
z = mu + std * torch.randn(1)

z.backward()
# torch.normal(mu, std)

In [4]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision
from torchvision.datasets import MNIST
from tqdm.auto import tqdm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from IPython.display import HTML

In [30]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)

### Prep Dataset ###
tensor_transforms = transforms.Compose(
    [
        transforms.Resize((28, 28)),
        transforms.ToTensor()
    ]
)

train_dataset = torchvision.datasets.MNIST(root=RAW_DATA_DIR / 'mnist', train=True, transform=tensor_transforms,
                                           download=True)
val_dataset = torchvision.datasets.MNIST(root=RAW_DATA_DIR / 'mnist', train=False, transform=tensor_transforms,
                                         download=True)
batch_size = 1024
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=6)
val_dl = torch.utils.data.DataLoader(val_dataset, batch_size=1024, shuffle=True, num_workers=6)

### Set Device ###
device = "cuda" if torch.cuda.is_available() else "cpu"

In [31]:
import math
class VAE(nn.Module):
    def __init__(self, latent_dim=2, input_size=28 * 28):
        super().__init__()
        self.input_size: int = input_size
        self.latent_dim: int = latent_dim

        self.encoder = nn.Sequential(
            nn.Linear(self.input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
        )
        self.fn_mu = nn.Linear(32, self.latent_dim)
        self.fn_logvar = nn.Linear(32, self.latent_dim)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_size),
            nn.Sigmoid()
        )
    def forward_dec(self, x):
        return self.decoder(x)

    def forward_enc(self, x):

        x = self.encoder(x)
        mu = self.fn_mu(x)
        logvar = self.fn_logvar(x)

        sigma = torch.exp(0.5 * logvar)
        noise = torch.randn_like(logvar, device=logvar.device)

        z = mu + sigma * noise
        return z, mu, logvar

    def forward(self, x):
        z, mu, logvar = self.forward_enc(x)

        return z, self.decoder(z), mu, logvar



In [32]:
def VAELoss(x, x_hat, mean, log_var, kl_weight=1, reconstruction_weight=1):
    pixel_mse = ((x - x_hat)**2)

    reconstruction_loss = pixel_mse.sum(axis=-1).mean()
    # reconstruction_loss = pixel_mse.mean()


    kl = (1 + log_var - mean**2 - torch.exp(log_var))

    kl_per_image = -0.5 * torch.sum(kl, dim=-1)


    kl_loss = torch.mean(kl_per_image)
    #print(reconstruction_loss, kl_loss)

    return reconstruction_loss * reconstruction_weight + kl_weight * kl_loss

x = torch.randn(4, 128)
x_hat = torch.randn(4, 128)

mean = torch.randn(4, 2)
log_var = torch.randn(4, 2)

VAELoss(x, x_hat, mean, log_var)


tensor(289.8410)

# Trainning

In [33]:
val_dataset.data.shape[1]

28

In [34]:
kl_weight = 1
input_size = val_dataset.data.shape[1]**2
epochs = 100
model = VAE(latent_dim=2, input_size=input_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
val_step = 10

torch.set_float32_matmul_precision('high')

train_loss = []

encoded_data_per_eval = []
train_losses =[]
val_losses =[]


model.train()

train = True

for epoch in tqdm(range(epochs)):
    train_loss_epoch = []

    for imgs, _ in train_dl:
        imgs = imgs.to(device)
        imgs = imgs.flatten(1)

        encoded, decoded, mu, logvar = model(imgs)

        loss = VAELoss(imgs, decoded, mu, logvar, kl_weight=kl_weight)
        train_loss_epoch.append(loss.item())

        optimizer.zero_grad()
        optimizer.step()
    avg_train_loss = np.mean(train_loss_epoch)


    if epoch % val_step == 0:
        model.eval()
        val_loss_epoch = []

        with torch.no_grad():
            for img, _ in val_dl:
                img = img.to(device)
                img = img.flatten(1)
                encoded, decoded, mu, logvar = model(img)
                loss = VAELoss(img, decoded, mu, logvar, kl_weight=kl_weight)
                val_loss_epoch.append(loss.item())

        avg_val_loss = np.mean(val_loss_epoch)

        print(f"Epoch {epoch} — train_loss: {avg_train_loss:.6f} — val_loss: {avg_val_loss:.6f}")
                # on sauvegarde les moyennes
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)

        model.train()





  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0 — train_loss: 181.805974 — val_loss: 181.868698
Epoch 10 — train_loss: 181.805516 — val_loss: 181.866783
Epoch 20 — train_loss: 181.804011 — val_loss: 181.871103
Epoch 30 — train_loss: 181.806826 — val_loss: 181.870471
Epoch 40 — train_loss: 181.805846 — val_loss: 181.872754
Epoch 50 — train_loss: 181.805054 — val_loss: 181.870107
Epoch 60 — train_loss: 181.804135 — val_loss: 181.868660
Epoch 70 — train_loss: 181.806805 — val_loss: 181.867168
Epoch 80 — train_loss: 181.803796 — val_loss: 181.867865
Epoch 90 — train_loss: 181.804073 — val_loss: 181.873997


VAE(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
  )
  (fn_mu): Linear(in_features=32, out_features=2, bias=True)
  (fn_logvar): Linear(in_features=32, out_features=2, bias=True)
  (decoder): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=784, bias=True)
    (7): Sigmoid()
  )
)